In [ ]:
# making sure that updates to imported files are immediately available without restarting the kernel
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

font = {"size": 15}
matplotlib.rc("font", **font)

sys.path.append(os.path.abspath(".."))
from utils import error_consistency, simulate_trials_from_copy_model, filter_df

## Significance testing

First, let's see how an EC-estimate obtained from an insufficient number of trials is not significant, while one from sufficiently many trials is.

In [ ]:
trials1 = np.array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1])
trials2 = np.array([0, 0, 0, 1, 0, 1, 1, 1, 1, 1])

ec, p_value = error_consistency(trials1, trials2)

print(ec, p_value)

In [ ]:
trials1, trials2 = simulate_trials_from_copy_model(
    kappa=0.1, acc1=0.7, acc2=0.75, n_trials=1000
)

ec, p_value = error_consistency(trials1, trials2)

print(ec, p_value)

## MvH

Testing the statistical significances of the MvH data, for clip and one human

In [ ]:
# load CLIP and subject-01 from Geirhos data
all_df = pd.read_parquet(
    "../geirhos_analysis/data/roberts_raw_data.parquet", engine="pyarrow"
)

df = all_df[all_df["subj"].isin(["subject-01", "clip"])]

df = filter_df(df)

ecs = []
pvals = []
exps = []
cons = []
for exp, exp_df in df.groupby("experiment", observed=True):
    for con, con_df in exp_df.groupby("condition", observed=True):

        ecdf = pd.merge(
            left=con_df[con_df["subj"] == "clip"],
            right=con_df[con_df["subj"] == "subject-01"],
            on="img_identifier",
        )

        ec, pval = error_consistency(
            ecdf["correct_x"].values, ecdf["correct_y"].values, n_bootstraps=50_000
        )

        ecs.append(ec)
        pvals.append(pval)
        exps.append(exp)
        cons.append(con)

res_df = pd.DataFrame(
    {"Experiment": exps, "Condition": cons, "EC": ecs, "p-value": pvals}
)

display(res_df)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
plt.grid(axis="y")
sns.scatterplot(data=res_df, x="EC", y="p-value", hue="Experiment", legend=True, ax=ax)
sns.despine()
ax.set_ylim(0, 0.35)
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
ax.axhline(0.05, color="red", linestyle="dotted")
plt.tight_layout()
plt.savefig("figures/pvalues.pdf", bbox_inches="tight")
plt.show()